In [1]:
#imports
import pandas as pd
from helper import pickle_store, pickle_restore
import dateutil
from adem import *
from collections import Counter 


# restore picke which was stored in email_fem.ipynb
#get email data of first 3 months: data is without date column converted to timedates type
email_raw_df = pickle_restore("pickle/email_rawdf_file")
usr_cmnty_map = pickle_restore("pickle/community_louvian_file")
usr_peer_map = pickle_restore("pickle/eid_role_map_file")
email_eid_map = pickle_restore("pickle/email_eid_map_file")





In [2]:

def find_intercmnty_num(x):
    user=x.user
    receipient_list=[]
    if (type(x.to) == str):
        # print (x.to.split(';'))
        receipient_list+=x.to.split(';')

    if (type(x.bcc) == str):
        # print (x.bcc.split(';'))
        receipient_list+=x.bcc.split(';')    

    if (type(x.cc) == str):
        # print (x.cc.split(';'))
        receipient_list+=x.cc.split(';')        

    # print (receipient_list)
    inter_cmnty_count=0
    intra_cmnty_count=0

    for u in receipient_list:
        if u in email_eid_map.keys():
            if usr_cmnty_map[user] == usr_cmnty_map[email_eid_map[u]]:
                intra_cmnty_count+=1
            else:
                inter_cmnty_count+=1
        else:
            inter_cmnty_count+=1

    # print(inter_cmnty_count, intra_cmnty_count)
    return inter_cmnty_count


def find_intracmnty_num(x):
    user=x.user
    receipient_list=[]
    if (type(x.to) == str):
        # print (x.to.split(';'))
        receipient_list+=x.to.split(';')

    if (type(x.bcc) == str):
        # print (x.bcc.split(';'))
        receipient_list+=x.bcc.split(';')    

    if (type(x.cc) == str):
        # print (x.cc.split(';'))
        receipient_list+=x.cc.split(';')        

    # print (receipient_list)
    inter_cmnty_count=0
    intra_cmnty_count=0

    for u in receipient_list:
        if u in email_eid_map.keys():
            if usr_cmnty_map[user] == usr_cmnty_map[email_eid_map[u]]:
                intra_cmnty_count+=1
            else:
                inter_cmnty_count+=1
        else:
            inter_cmnty_count+=1

    # print(inter_cmnty_count, intra_cmnty_count)
    return intra_cmnty_count

#get email_raw_df with intercmnty_freq and intracmnty_freq columns
email_raw_df['intercmnty_freq']=email_raw_df.apply(lambda x: find_intercmnty_num(x), axis = 1)   
email_raw_df['intracmnty_freq']=email_raw_df.apply(lambda x: find_intracmnty_num(x), axis = 1)   


In [3]:
email_freq_feature_df=email_raw_df[['user', 'date', 'intercmnty_freq', 'intracmnty_freq']]

email_freq_feature_df['date'] = pd.to_datetime(email_freq_feature_df['date'])
mask = (email_freq_feature_df['date'] <= end_d) & (email_freq_feature_df['date'] >= start_d)
email_freq_feature_df = email_freq_feature_df.loc[mask]

print("done heavy part...")



#create a copy of orig df to operate on deep copied dataframe for UBP , PBP and CBP
email_freq_feature_cmnty_df = email_freq_feature_df.copy()
email_freq_feature_peer_df = email_freq_feature_df.copy()

# 1. in email_freq_feature_df add community and peer column with id as values.
email_freq_feature_cmnty_df['cmnty']=email_freq_feature_cmnty_df.apply(lambda row: usr_cmnty_map[row.user], axis=1)

email_freq_feature_peer_df['peer']=email_freq_feature_peer_df.apply(lambda row: usr_peer_map[row.user], axis=1)




done heavy part...


In [4]:
# Convert date back to string format
email_freq_feature_df['date']=email_freq_feature_df['date'].astype(str)
email_freq_feature_cmnty_df['date']=email_freq_feature_cmnty_df['date'].astype(str)
email_freq_feature_peer_df['date']=email_freq_feature_peer_df['date'].astype(str)


In [5]:
#now the df of form email_raw_df[['user', 'date', 'intercmnty_freq', 'intracmnty_freq']] is ready use groupby to prepare dic for anomaly detection.inter_cmnty_count
#heavy operations
def prep_ubp_dic(df, inter_cmnty= True):
    #convert date column of srt type to Timestamp type.
    df['date'] = df['date'].apply(dateutil.parser.parse, dayfirst=True)
    #now convert date column of Timestamptype to only date values and remove hr:mm:ss
    df['date'] = pd.to_datetime(df['date']).dt.date

    #creates user, date wise groups and each internal group is a dataframe.
    # grp=tmp_email_freq_feature_df.groupby(['user', 'date'])

    # create user, date wise groups and count unique dates then add a separate column for the counts
    if inter_cmnty:
        df=df.groupby(['user', 'date']).intercmnty_freq.agg('sum').to_frame('outcmnty_ef').reset_index()
    else:
        df=df.groupby(['user', 'date']).intracmnty_freq.agg('sum').to_frame('incmnty_ef').reset_index()

    #groupby user now To populate a dictionary for each user.
    grp=df.groupby(['user'])

    email_freq_feature_dic={} #user:df with feature value
    #iterating groups
    for name, group in grp:
        # print (name)
        # print (group)
        email_freq_feature_dic[name]=group
        # ldf = group.groupby(['date'], as_index=False)['cntr'].size()

    return email_freq_feature_dic

intercmnty_df = email_freq_feature_df.copy()
intracmnty_df = email_freq_feature_df.copy()

ef_ubp_intercmnty_feature_dic = prep_ubp_dic(intercmnty_df, inter_cmnty= True)
ef_ubp_intracmnty_feature_dic = prep_ubp_dic(intracmnty_df, inter_cmnty= False)

In [6]:
def prep_cbp_dic(df, inter_cmnty= True):
    #convert date column of srt type to Timestamp type.
    df['date'] = df['date'].apply(dateutil.parser.parse, dayfirst=True)
    #now convert date column of Timestamptype to only date values and remove hr:mm:ss
    df['date'] = pd.to_datetime(df['date']).dt.date

    #creates user, date wise groups and each internal group is a dataframe.
    # grp=tmp_email_freq_feature_df.groupby(['user', 'date'])

    # create user, date wise groups and count unique dates then add a separate column for the counts
    if inter_cmnty:
        df=df.groupby(['cmnty', 'date']).intercmnty_freq.agg('sum').to_frame('outcmnty_ef').reset_index()
        cmnty_len = Counter(usr_cmnty_map.values())
        #make values of email freq avg of community
        df['outcmnty_ef'] = df.apply( lambda x: x.outcmnty_ef/cmnty_len[x.cmnty], axis=1)
    else:
        df=df.groupby(['cmnty', 'date']).intracmnty_freq.agg('sum').to_frame('incmnty_ef').reset_index()
        cmnty_len = Counter(usr_cmnty_map.values())
        #make values of email freq avg of community
        df['incmnty_ef'] = df.apply( lambda x: x.incmnty_ef/cmnty_len[x.cmnty], axis=1)


    #groupby user now To populate a dictionary for each user.
    grp=df.groupby(['cmnty'])

    ef_feature_dic={} #user:df with feature value
    #iterating groups
    for name, group in grp:
        # print (name)
        # print (group)
        ef_feature_dic[name]=group
        # ldf = group.groupby(['date'], as_index=False)['cntr'].size()

    return ef_feature_dic    

intercmnty_cbp_df = email_freq_feature_cmnty_df.copy()
intracmnty_cbp_df = email_freq_feature_cmnty_df.copy()

ef_cbp_intercmnty_feature_dic = prep_cbp_dic(intercmnty_cbp_df, inter_cmnty= True)
ef_cbp_intracmnty_feature_dic = prep_cbp_dic(intracmnty_cbp_df, inter_cmnty= False)    

In [7]:
def prep_pbp_dic(df, inter_cmnty= True):
    #convert date column of srt type to Timestamp type.
    df['date'] = df['date'].apply(dateutil.parser.parse, dayfirst=True)
    #now convert date column of Timestamptype to only date values and remove hr:mm:ss
    df['date'] = pd.to_datetime(df['date']).dt.date

    #creates user, date wise groups and each internal group is a dataframe.
    # grp=tmp_email_freq_feature_df.groupby(['user', 'date'])

    # create user, date wise groups and count unique dates then add a separate column for the counts
    if inter_cmnty:
        df=df.groupby(['peer', 'date']).intercmnty_freq.agg('sum').to_frame('outcmnty_ef').reset_index()
        peer_len = Counter(usr_peer_map.values())
        #make values of email freq avg of community
        df['outcmnty_ef'] = df.apply( lambda x: x.outcmnty_ef/peer_len[x.peer], axis=1)
    else:
        df=df.groupby(['peer', 'date']).intracmnty_freq.agg('sum').to_frame('incmnty_ef').reset_index()
        peer_len = Counter(usr_peer_map.values())
        #make values of email freq avg of community
        df['incmnty_ef'] = df.apply( lambda x: x.incmnty_ef/peer_len[x.peer], axis=1)


    #groupby user now To populate a dictionary for each user.
    grp=df.groupby(['peer'])

    ef_feature_dic={} #user:df with feature value
    #iterating groups
    for name, group in grp:
        # print (name)
        # print (group)
        ef_feature_dic[name]=group
        # ldf = group.groupby(['date'], as_index=False)['cntr'].size()

    return ef_feature_dic    

intercmnty_pbp_df = email_freq_feature_peer_df.copy()
intracmnty_pbp_df = email_freq_feature_peer_df.copy()

ef_pbp_intercmnty_feature_dic = prep_pbp_dic(intercmnty_pbp_df, inter_cmnty= True)
ef_pbp_intracmnty_feature_dic = prep_pbp_dic(intracmnty_pbp_df, inter_cmnty= False)

In [8]:
# ef_ubp_intercmnty_feature_dic = prep_ubp_dic(intercmnty_df, inter_cmnty= True)
# ef_ubp_intracmnty_feature_dic = prep_ubp_dic(intracmnty_df, inter_cmnty= False)

# ef_cbp_intercmnty_feature_dic = prep_cbp_dic(intercmnty_cbp_df, inter_cmnty= True)
# ef_cbp_intracmnty_feature_dic = prep_cbp_dic(intracmnty_cbp_df, inter_cmnty= False)    

# ef_pbp_intercmnty_feature_dic = prep_pbp_dic(intercmnty_pbp_df, inter_cmnty= True)
# ef_pbp_intracmnty_feature_dic = prep_pbp_dic(intracmnty_pbp_df, inter_cmnty= False)

#store all preped dic
pickle_store("pickle/f2/f2_ubp_file", ef_ubp_intercmnty_feature_dic) #raw means not indexed by Timeindex
pickle_store("pickle/f2/f2_pbp_file", ef_pbp_intercmnty_feature_dic) #raw means not indexed by Timeindex
pickle_store("pickle/f2/f2_cbp_file", ef_cbp_intercmnty_feature_dic) #raw means not indexed by Timeindex


pickle_store("pickle/f3/f3_ubp_file", ef_ubp_intracmnty_feature_dic) #raw means not indexed by Timeindex
pickle_store("pickle/f3/f3_pbp_file", ef_pbp_intracmnty_feature_dic) #raw means not indexed by Timeindex
pickle_store("pickle/f3/f3_cbp_file", ef_cbp_intracmnty_feature_dic) #raw means not indexed by Timeindex



In [ ]:
ef_ubp_intercmnty_feature_dic = pickle_restore("pickle/f2/f2_ubp_file") #raw means not indexed by Timeindex
ef_pbp_intercmnty_feature_dic = pickle_restore("pickle/f2/f2_pbp_file") #raw means not indexed by Timeindex
ef_cbp_intercmnty_feature_dic = pickle_restore("pickle/f2/f2_cbp_file") #raw means not indexed by Timeindex


ef_ubp_intracmnty_feature_dic = pickle_restore("pickle/f3/f3_ubp_file") #raw means not indexed by Timeindex
ef_pbp_intracmnty_feature_dic = pickle_restore("pickle/f3/f3_pbp_file") #raw means not indexed by Timeindex
ef_cbp_intracmnty_feature_dic = pickle_restore("pickle/f3/f3_cbp_file") #raw means not indexed by Timeindex


In [ ]:
# f2 intercmnty

count = 0
intercmnty_fname = 'outcmnty_ef'
intracmnty_fname = 'incmnty_ef'

ubp_events_dic={}
for key in ef_ubp_intercmnty_feature_dic:
    usr=key
    debug_df = ef_ubp_intercmnty_feature_dic[usr]


    #do a depp copy of preped feature dataframe
    df = debug_df.copy()
    anom_ev=anom_calc_ubp(df, intercmnty_fname, usr, ws=10, sig=3)
    
    if len(anom_ev['anomalies_dict']):
        ubp_events_dic[key]=anom_ev
    
    # count += 1
    # if count%10 == 0:
    #     break

cbp_events_dic={}
for key in ef_ubp_intercmnty_feature_dic:
    usr=key
    debug_df = ef_ubp_intercmnty_feature_dic[usr]
    debug_cmnty_df = ef_cbp_intercmnty_feature_dic[usr_cmnty_map[usr]]


    #do a depp copy of preped feature dataframe
    df = debug_df.copy()
    df_cmnty = debug_cmnty_df.copy()

    anom_ev=anom_calc_cbp(df, df_cmnty, intercmnty_fname, usr, ws=10, sig=3)

    if len(anom_ev['anomalies_dict']):
        cbp_events_dic[key]=anom_ev


    # count += 1
    # if count%10 == 0:
    #     break
pbp_events_dic={}
for key in ef_ubp_intercmnty_feature_dic:
    usr=key
    debug_df = ef_ubp_intercmnty_feature_dic[usr]
    debug_peer_df = ef_pbp_intercmnty_feature_dic[usr_peer_map[usr]]


    #do a depp copy of preped feature dataframe
    df = debug_df.copy()
    df_peer = debug_peer_df.copy()

    anom_ev=anom_calc_pbp(df, df_peer, intercmnty_fname, usr, ws=10, sig=3)

    if len(anom_ev['anomalies_dict']):
        pbp_events_dic[key]=anom_ev
    
    # count += 1
    # if count%10 == 0:
    #     break


In [ ]:
def get_anom_list(events_dic):
    anom_user_lst=[]
    for anom_usr in events_dic:
        a_ev = events_dic[anom_usr]
        # print('dict a_ev=', a_ev['anomalies_dict'])
        for k, v in a_ev['anomalies_dict'].items():
            if v != 0:
                anom_user_lst.append(anom_usr)
                break
    return anom_user_lst


f2_anom_user_lst=[]
# for anom_usr in ubp_events_dic:
#     a_ev = ubp_events_dic[anom_usr]
#     print('dict a_ev=', a_ev['anomalies_dict'])
#     for k, v in a_ev['anomalies_dict'].items():
#         if v != 0:
#             f1_anom_user_lst.append(anom_usr)
#             break

f2_anom_user_lst += get_anom_list(ubp_events_dic)
f2_anom_user_lst += get_anom_list(pbp_events_dic)
f2_anom_user_lst += get_anom_list(cbp_events_dic)

len(set(f2_anom_user_lst))

# save list of anom users
pickle_store("pickle/f2/f2_anom_set_file", set(f2_anom_user_lst)) #raw means not indexed by Timeindex



In [ ]:
ubp_events_dic

In [25]:
######PBP f3 intra

count = 0
intercmnty_fname = 'outcmnty_ef'
intracmnty_fname = 'incmnty_ef'

pbp_events_dic={}
for key in ef_ubp_intracmnty_feature_dic:
    usr=key
    debug_df = ef_ubp_intracmnty_feature_dic[usr]
    debug_peer_df = ef_pbp_intracmnty_feature_dic[usr_peer_map[usr]]


    #do a depp copy of preped feature dataframe
    df = debug_df.copy()
    df_peer = debug_peer_df.copy()

    anom_ev = anom_calc_pbp(df, df_peer, intracmnty_fname, usr, ws=10, sig=3)

    if len(anom_ev['anomalies_dict']):
        pbp_events_dic[key]=anom_ev

    
    # count += 1
    # if count%20 == 0:
    #     break        

cbp_events_dic={}
for key in ef_ubp_intracmnty_feature_dic:
    usr=key
    debug_df = ef_ubp_intracmnty_feature_dic[usr]
    debug_cmnty_df = ef_cbp_intracmnty_feature_dic[usr_cmnty_map[usr]]


    #do a depp copy of preped feature dataframe
    df = debug_df.copy()
    df_cmnty = debug_cmnty_df.copy()

    anom_ev = anom_calc_cbp(df, df_cmnty, intracmnty_fname, usr, ws=10, sig=3)
    if len(anom_ev['anomalies_dict']):
        cbp_events_dic[key]=anom_ev

    
    # count += 1
    # if count%20 == 0:
    #     break        
ubp_events_dic={}
for key in ef_ubp_intracmnty_feature_dic:
    usr=key
    debug_df = ef_ubp_intracmnty_feature_dic[usr]


    #do a depp copy of preped feature dataframe
    df = debug_df.copy()
    anom_ev = anom_calc_ubp(df, intracmnty_fname, usr, ws=10, sig=3)
    if len(anom_ev['anomalies_dict']):
        ubp_events_dic[key]=anom_ev

    
    # count += 1
    # if count%20 == 0:
    #     break    

In [27]:
f3_anom_user_lst=[]
# for anom_usr in ubp_events_dic:
#     a_ev = ubp_events_dic[anom_usr]
#     print('dict a_ev=', a_ev['anomalies_dict'])
#     for k, v in a_ev['anomalies_dict'].items():
#         if v != 0:
#             f1_anom_user_lst.append(anom_usr)
#             break

f3_anom_user_lst += get_anom_list(ubp_events_dic)
f3_anom_user_lst += get_anom_list(pbp_events_dic)
f3_anom_user_lst += get_anom_list(cbp_events_dic)

len(set(f3_anom_user_lst))

# save list of anom users
pickle_store("pickle/f3/f3_anom_set_file", set(f3_anom_user_lst)) #raw means not indexed by Timeindex


In [31]:
len(set(cbp_events_dic))


649